In [ ]:
import pandas as pd
import numpy as np
import os

from pytradersan import constants
from pytradersan.helpers import standardize_transactions
from pytradersan.portfolio import Portfolio
from pytradersan.schwab_api import (
    SCHWAB_API_URL,
    SCHWAB_API_TRANSACTION_TYPES, 
    API_COLUMN_MAPPER, 
    get_combined_transactions
)

from pathlib import Path
import glob
from collections import defaultdict
import requests
import json
from tzlocal import get_localzone

from dotenv import load_dotenv, find_dotenv

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
load_dotenv(find_dotenv())
SCHWAB_BEARER_TOKEN = os.getenv("SCHWAB_BEARER_TOKEN")

In [ ]:
transactions = get_combined_transactions(
    base_url=SCHWAB_API_BASE_URL,
    token=SCHWAB_BEARER_TOKEN,
    start_date='2024-01-01',
)

In [ ]:
trades = transactions['TRADE'].apply(
    lambda x: parse_trades(x['transferItems']), axis=1
).apply(pd.Series)
trades['price'] = trades['price'].astype(float)
trades['quantity'] = trades['quantity'].astype(float)
trades['symbol'] = trades['symbol'].astype(str)
trades['asset'] = trades['asset'].astype(str)
trades['action'] = np.where(
    trades['quantity'] > 0, 'BUY', 'SELL'
)
trades['action'] = trades['action'].astype(str)
trades = trades.join(
    transactions['TRADE'].copy(deep=True).drop(columns=['transferItems'])
)
trades = trades.rename(columns=API_COLUMN_MAPPER)
trades = trades[API_COLUMN_MAPPER.values()]
trades = trades.drop_duplicates()

In [ ]:
trades.sort_values('date')


In [ ]:
transactions['TRADE']

In [ ]:
constants.TRANSACTIONS_STANDARD_COLS

In [ ]:
df = transactions['TRADE'].drop('transferItems', axis=1).join(transactions['TRADE']['transferItems'].apply(parse_trades).apply(pd.Series))

In [ ]:
origin = Path('../..')

schwab_files = glob.glob(str(origin / 'data' / 'schwab*.csv'))
print(schwab_files)

# Load raw Schwab transactions
schwab_transactions = defaultdict(dict)
for schwab_file in schwab_files:
    name = schwab_file.split('/')[-1].split('.csv')[0]
    print(f'Loading {schwab_file} with key {name}')
    schwab_transactions[name] = pd.read_csv(schwab_file)
